In [1]:
from argparse import ArgumentParser
import yaml
import numpy as np
from skimage.io import imread
from skimage.morphology import binary_dilation, disk
from skimage.measure import regionprops, label
import os
import copy
from glob import glob
import pickle
from tqdm import tqdm
import matplotlib.pyplot as plt
import scipy
import pandas as pd

In [ ]:
def getid(samp_path):
    return samp_path.split("/")[-1].split(".")[0]

In [ ]:
def create_pmask(segpath, r):
    #print("create pmask", segpath)
    seg = imread(segpath)
    mask = np.zeros_like(seg)
    rps = regionprops(seg)
    centroids = np.array(
        list(map(lambda x : np.array(x.centroid).astype(int), rps))
    )
    mask[centroids[:,0], centroids[:,1]] = 1
    mask = binary_dilation(mask, disk(r))
    print("mask created")
    return mask

In [ ]:
def compute_masks(sid, radii, methods, datapath):
    masks = {}
    for r in radii:
        rmasks = {}
        print('Radius', r)
        for m in methods:
            print("methods", m)
            #print(glob(os.path.join(datapath, m, sid + "*")))
            segpath = glob(os.path.join(datapath, m, sid + "*"))[0]
            #print("compute pmask", segpath)
            pmask = create_pmask(segpath, r)
            rmasks[m] = pmask
        stack = np.stack(list(rmasks.values()))
        avg_mask = stack.mean(0)
        rmasks["mean"] = avg_mask
        masks[r] = rmasks
        print("Masks stacked")
    return masks

In [ ]:
def write_pmasks(sample_ids, radii, methods, datapath, save_dir):
    print(f"Writing probability masks for {len(sample_ids)} samples")
    for sid in tqdm(sample_ids):
        save_path = os.path.join(save_dir, f"{sid}.pkl")
        if os.path.exists(save_path):
            continue
            
        #print("sid", sid)

        sid_masks = compute_masks(sid, radii, methods, datapath)
        with open(save_path, "wb") as handle:
            pickle.dump(sid_masks, handle)
    print(f"Proability masks saved to {save_dir}")



In [ ]:
def filter_mask(mask, avg_labs):
    filtered = copy.deepcopy(mask)
    mask_lab = label(mask)
    rps = regionprops(mask_lab)
    for rp in rps:
        coords = rp.coords
        vals = avg_labs[coords[:,0], coords[:,1]]
        uniq, counts = np.unique(vals, return_counts=True)
        if uniq[0] == 0:
            uniq = uniq[1:]
            counts = counts[1:]
        n_unique = len(uniq)
        if n_unique > 1:
            amax = np.argmax(counts)
            top_val = uniq[amax]
            idxs = np.where(vals != top_val)
            to_zero = coords[idxs,:][0]
            filtered[to_zero[:,0], to_zero[:,1]] = False
    return filtered

In [ ]:
def filter_pmasks(sample_ids, pmask_save_dir, filtered_save_dir, min_num_agree, methods):
    print(f"Filtering probability masks for {len(sample_ids)} samples")
    for sid in tqdm(sample_ids):
        with open(os.path.join(pmask_save_dir, f"{sid}.pkl"), "rb") as handle:
            print(os.path.join(pmask_save_dir, f"{sid}.pkl"))
            data = pickle.load(handle)

        filtered_masks = {}
        for r, masks in data.items():
            avg = masks["mean"]
            avg_threshd = (avg >= (min_num_agree / len(methods)))
            avg_labs = label(avg_threshd)

            r_filtered_masks = {}
            for m in methods:
                r_filtered_masks[m] = filter_mask(masks[m], avg_labs)

            new_stack = np.stack(list(r_filtered_masks.values()))        
            new_avg = new_stack.mean(0)
            r_filtered_masks["mean"] = new_avg
            filtered_masks[r] = r_filtered_masks
            
        with open(os.path.join(filtered_save_dir, f"{sid}.pkl"), "wb") as handle:
            pickle.dump(filtered_masks, handle)
            
    print(f"Filtered probability masks saved to {filtered_save_dir}")


In [ ]:
def eval_mask(gt, m):
    rps = regionprops(m)
    coords = list(map(lambda x : x.coords, rps))
    correct = 0
    
    for c in coords:
        correct += (gt[c[:,0], c[:,1]]).max()
    precision = correct / len(rps)
    
    gt_labs = label(gt)
    #print(gt_labs)
    gt_rps = regionprops(gt_labs)
    #print(gt_rps)
    coords = list(map(lambda x : x.coords, gt_rps))
    correct = 0
    
    for c in coords:
        correct += (m[c[:,0], c[:,1]]).max() > 0
    recall = correct / len(gt_rps)
    
    assert precision <= 1
    assert precision >= 0
    assert recall <= 1
    assert recall >= 0
    
    return precision, recall

In [ ]:
def evaluate_masks(sample_ids, filtered_pmask_save_dir, radii, min_num_agree, num_methods):
    precision = {}
    recall = {}
    print(f"Computing precision and recall for {len(sample_ids)} samples")
    for sid in tqdm(sample_ids):
        data_load_path = os.path.join(filtered_pmask_save_dir, f"{sid}.pkl")
        with open(data_load_path, "rb") as handle:
            data = pickle.load(handle)

        sid_precisions = dict((r, {}) for r in radii)
        sid_recalls = dict((r, {}) for r in radii)

        for r, masks in data.items():

            avg = masks["mean"]
            avg_thresh = (avg >= (min_num_agree / num_methods))
            
            for name, mask in masks.items():
                if name == "mean":
                    continue
                labd_mask = label(mask)
                prec, rec = eval_mask(avg_thresh, labd_mask)
                sid_precisions[r][name] = prec
                sid_recalls[r][name] = rec

        precision[sid] = sid_precisions
        recall[sid] = sid_recalls

    return precision, recall

In [ ]:
def main():
    #parser = ArgumentParser()
    #parser.add_argument("--config", type=str, default="./config.yml", help="Path to config file")
    #parser.add_argument("--compute-pmasks", action="store_true", help="Compute probability masks")
    #parser.add_argument("--filter-pmasks", action="store_true", help="Filter probability masks")
    #parser.add_argument("--compute-scores", action="store_true", help="Compute scores")
    #args = parser.parse_args()
    
    # m - mesmer, s- startdist, c - cellpose, u - unet, r - MaskRCNN, x - UnMicst

    #with open(args.config, "r") as handle:
    #    config = yaml.load(handle, Loader=yaml.FullLoader)
    methods =  ["Mesmer", "Stardist", "Cellpose", "UnMicst"] #config["methods"]
    radii = [4, 8, 12, 16, 20, 24, 28, 32] #config["radii"]
    num_agree = 3 #config["num_agree"]
    datapath = "/home/groups/ChangLab/dharani/OHSU-TMA/Segmentations" #"/home/groups/ChangLab/dataset/HMS-TMA-TNP/DATA-03292022" # #config["datapath"]
    results_dir = "/home/groups/ChangLab/dharani/OHSU-TMA/results_mscx" #config["resultsdir"]
    
    compute_pmasks = False
    filter_pmask = True
    compute_scores = True

    if not os.path.exists(results_dir):
        os.makedirs(results_dir)
    pmask_save_dir = os.path.join(results_dir, "pmasks")
    if not os.path.exists(pmask_save_dir):
        os.makedirs(pmask_save_dir)

    sample_ids = sorted(os.listdir(os.path.join(datapath, methods[0])))
    sample_ids = [s.split(".")[0] for s in sample_ids]
    #sample_ids = [s.split("_")[0] for s in sample_ids] # only for the groundtruth
    sample_ids = sample_ids[0:1]
    print("sample_ids", sample_ids)
    

    if compute_pmasks:
        write_pmasks(sample_ids, radii, methods, datapath, pmask_save_dir)

    filtered_pmask_save_dir = os.path.join(results_dir, "filtered_pmasks")
    if not os.path.exists(filtered_pmask_save_dir):
        os.makedirs(filtered_pmask_save_dir)

    if filter_pmask:
        filter_pmasks(sample_ids, pmask_save_dir, filtered_pmask_save_dir, num_agree, methods)

    precision_scores_path = os.path.join(results_dir, "precision_scores.pkl")
    recall_scores_path = os.path.join(results_dir, "recall_scores.pkl")

    if compute_scores:
        precision, recall = evaluate_masks(sample_ids, filtered_pmask_save_dir, radii, num_agree, len(methods))
        with open(precision_scores_path, "wb") as handle:
            pickle.dump(precision, handle)
            print(f"Saved precision scores to {precision_scores_path}")
        with open(recall_scores_path, "wb") as handle:
            pickle.dump(recall, handle)
            print(f"Saved recall scores to {recall_scores_path}")


if __name__ == '__main__':
    main()

### Adjusted Ratios

In [37]:
# done - radius 2, 4, 6, 8, 10, 12, 14
overall_scores = {0.945:[0.943, 0.942, 0.945, 0.948, 0.948, 0], # these values are from evaluation_quantification excel sheet
                  0.935:[0.929, 0.935, 0.936, 0.936, 0, 0.942],
                  0.934:[0.933, 0.938, 0.929, 0, 0.936, 0.936],
                  0.916:[0.917, 0.918, 0, 0.911, 0.916, 0.918],
                  0.892:[0.887, 0, 0.894, 0.896, 0.892, 0.891],
                  0.875:[0, 0.870, 0.877, 0.876, 0.873, 0.879]}

value_change = []

for key, value in overall_scores.items():
    temp = []
    for i in value:
        #print(key)
        temp.append( ((i-key)/key)*100 )
    value_change.append(temp)
    
df = pd.DataFrame(value_change)
df = df.replace(-100.000000, np.NaN)
df = df.round(3)
df.loc[len(df.index)] = df.mean(axis=0, skipna=True)
df = df.round(4)
display(df)    

,0,1,2,3,4,5
0,-0.2120,-0.3170,0.000,0.317,0.3170,NaN
1,-0.6420,0.0000,0.107,0.107,NaN,0.7490
2,-0.1070,0.4280,-0.535,NaN,0.2140,0.2140
3,0.1090,0.2180,NaN,-0.546,0.0000,0.2180
4,-0.5610,NaN,0.224,0.448,0.0000,-0.1120
5,NaN,-0.5710,0.229,0.114,-0.2290,0.4570
6,-0.2826,-0.0484,0.005,0.088,0.0604,0.3052


In [38]:
x = df.iloc[-1].tolist()
print(x) #overall

from scipy.special import softmax

print(softmax(x))

[-0.2826, -0.0484, 0.005, 0.088, 0.0604, 0.3052]
[0.12113992 0.15310841 0.16150663 0.17548371 0.17070658 0.21805476]


In [ ]:
#config = {"Mesmer":{4:0.45, 8:0.05, 12:0.40, 16:0.10}, "Stardist":{4:0.04, 8:0.59, 12:0.06, 16:0.31}}

#for key, value in config.items():
#    for radius, ratio in value.items():
#        if key == "Stardist" and radius == 4:
#            print(ratio)

In [ ]:
2 : [0.15708386 0.16780117 0.15878956 0.16786831 0.17760816 0.17084894]
4 : [0.15110248 0.15107227 0.14010026 0.18481555 0.21399621 0.15891323]
6 : [0.08883816 0.09650707 0.15456524 0.12052007 0.29032945 0.24924]
8 : [0.09821838 0.08273092 0.13536751 0.20905416 0.20934704 0.26528199]
10 : [0.13989789 0.11099807 0.16678621 0.18547365 0.14482307 0.2520211 ]
12 : [0.1258211  0.12931626 0.16634452 0.18505647 0.14180616 0.25165549]
14 : [0.12531543 0.14245611 0.17150869 0.16851655 0.1434568  0.24874642]
16 : [0.12113992 0.15310841 0.16150663 0.17548371 0.17070658 0.21805476]

In [40]:
weights_array = [[0.15708386, 0.16780117, 0.15878956, 0.16786831, 0.17760816, 0.17084894],
                 [0.15110248, 0.15107227, 0.14010026, 0.18481555, 0.21399621, 0.15891323],
                 [0.08883816, 0.09650707, 0.15456524, 0.12052007, 0.29032945, 0.24924],
                 [0.09821838, 0.08273092, 0.13536751, 0.20905416, 0.20934704, 0.26528199],
                 [0.13989789, 0.11099807, 0.16678621, 0.18547365, 0.14482307, 0.2520211 ],
                 [0.1258211,  0.12931626, 0.16634452, 0.18505647, 0.14180616, 0.25165549],
                 [0.12531543, 0.14245611, 0.17150869, 0.16851655, 0.1434568,  0.24874642],
                 [0.12113992, 0.15310841, 0.16150663, 0.17548371, 0.17070658, 0.21805476]]

In [59]:
weights_array_transpose = np.transpose(weights_array)

radius = [2, 4, 6, 8, 10, 12, 14, 16]
methods = ["unmicst", "unet", "cellpose", "maskrcnn", "stardist", "mesmer"]
weights_dict = {}

weights_df = pd.DataFrame(weights_array_transpose)
#display(weights_df)

In [57]:
for i in range(6):
    temp_list = weights_df.iloc[i].tolist()
    temp_dict = {}
    for j in range(len(temp_list)):
        temp_dict[radius[j]] = temp_list[j]
    weights_dict[methods[i]] = temp_dict

In [58]:
weights_dict

{'unmicst': {2: 0.15708386,
  4: 0.15110248,
  6: 0.08883816,
  8: 0.09821838,
  10: 0.13989789,
  12: 0.1258211,
  14: 0.12531543,
  16: 0.12113992},
 'unet': {2: 0.16780117,
  4: 0.15107227,
  6: 0.09650707,
  8: 0.08273092,
  10: 0.11099807,
  12: 0.12931626,
  14: 0.14245611,
  16: 0.15310841},
 'cellpose': {2: 0.15878956,
  4: 0.14010026,
  6: 0.15456524,
  8: 0.13536751,
  10: 0.16678621,
  12: 0.16634452,
  14: 0.17150869,
  16: 0.16150663},
 'maskrcnn': {2: 0.16786831,
  4: 0.18481555,
  6: 0.12052007,
  8: 0.20905416,
  10: 0.18547365,
  12: 0.18505647,
  14: 0.16851655,
  16: 0.17548371},
 'stardist': {2: 0.17760816,
  4: 0.21399621,
  6: 0.29032945,
  8: 0.20934704,
  10: 0.14482307,
  12: 0.14180616,
  14: 0.1434568,
  16: 0.17070658},
 'mesmer': {2: 0.17084894,
  4: 0.15891323,
  6: 0.24924,
  8: 0.26528199,
  10: 0.2520211,
  12: 0.25165549,
  14: 0.24874642,
  16: 0.21805476}}